### Note:

This notebook is for creating the dataset for NN model.

In [7]:
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset
import rasterio
from torchvision import transforms

In [42]:
class Pluvial_Dataset(Dataset):
    def __init__(self, rainfall_path, flood_folder, building_path, curve_path, DEM_path, time_of_concentration, threshold = 0.3):
        # Load rainfall data
        df = pd.read_excel(rainfall_path, header=0)
        rainfall_series = df.drop(columns=["Time"]).astype(float).reset_index(drop=True)
        self.rainfall = [torch.tensor(rainfall_series.iloc[:, i].values.astype(np.float32)).unsqueeze(1)
                         for i in range(rainfall_series.shape[1])]
        
        # Calculate rainfall durations
        self.durations = []
        for r in self.rainfall:
            r_np = r.squeeze().numpy()
            non_zero = np.nonzero(r_np)[0]
            duration = 0 if len(non_zero) == 0 else (non_zero[-1] - non_zero[0] + 1)
            self.durations.append(torch.tensor([duration], dtype=torch.float32))
        
        # Load building, curve number, and DEM rasters (shared for all events)
        with rasterio.open(building_path) as src:
            self.buildings = torch.tensor(src.read(1).astype(np.float32), dtype=torch.float32).unsqueeze(0)
        with rasterio.open(curve_path) as src:
            self.curve = torch.tensor(src.read(1).astype(np.float32), dtype=torch.float32).unsqueeze(0)
        with rasterio.open(DEM_path) as src:
            self.DEM = torch.tensor(src.read(1).astype(np.float32), dtype=torch.float32).unsqueeze(0)

        self.flood_folder = flood_folder
        self.time_of_concentration = torch.tensor([time_of_concentration], dtype=torch.float32)

        self.transform = transforms.Compose([
            transforms.ToTensor()
        ])

        self.threshold = threshold
    
    def __len__(self):
        return len(self.rainfall)
    
    def __getitem__(self, idx):
        # Rainfall sequence
        rain = self.rainfall[idx]
        dur = self.durations[idx]
        tc = self.time_of_concentration
        curve = self.curve
        DEM = self.DEM
        build = self.buildings

        # Load corresponding flood raster
        event_folder = os.path.join(self.flood_folder, f"{idx}")
        flood_path = os.path.join(event_folder, f"{idx}DepthFLT", f"{idx}Depth.flt")
        with rasterio.open(flood_path) as src:
            flood = torch.tensor(src.read(1).astype(np.float32), dtype=torch.float32).unsqueeze(0)
        flood[flood > 5] = 0
        flood = (flood > self.threshold).to(torch.uint8)

        return {
            "rainfall": rain,
            "duration": dur,
            "tc": tc,
            "curve": curve,
            "DEM": DEM,
            "buildings": build,
            "flood": flood
        }

In [43]:
rainfall_path = "Data_Sample/synthetic_rainfall_variable_duration_2.xlsx"
flood_folder = "Data_Sample/PCSWMM_Output_Files"
building_path = "Data_Sample/PCSWMM_Output_Files/1/1DepthFLT/1Depth.FLT"
curve_path = "Data_Sample/PCSWMM_Output_Files/1/1DepthTiff/1DepthT.TIF"
DEM_path = "Data_Sample/PCSWMM_Output_Files/1/1DepthTiff/1DepthT.TIF"
time_of_concentration = 3.5  # it is an example

In [44]:
dataset = Pluvial_Dataset(
    rainfall_path=rainfall_path,
    flood_folder=flood_folder,
    building_path=building_path,
    curve_path=curve_path,
    DEM_path = DEM_path,
    time_of_concentration=time_of_concentration
)

In [45]:
dataset[1]

{'rainfall': tensor([[0.0000],
         [0.0000],
         [0.0629],
         [0.1566],
         [0.0968],
         [0.0825],
         [0.0352],
         [0.0078],
         [0.0883],
         [0.0000],
         [0.0021],
         [0.0000],
         [0.0592],
         [0.0000],
         [0.0000],
         [0.0974],
         [0.2079],
         [0.3912],
         [0.2215],
         [0.0723],
         [0.0672],
         [0.0284],
         [0.0299],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
